In [1]:
import pandas as pd
import numpy as np

res = pd.read_csv("final_scores_for_plot.csv", index_col = 0)
res.head()

,gameId,gamesPlayed,teamAbbrev,opponentTeamAbbrev,goalsFor,goalsAgainst,gameLocationCode,wins,ties,shootoutGamesWon,year,seasonId,homeScore,roadScore,matchup,homeTeam,awayTeam,finalScore
0,1919020014,1,SEN,QBD,2.0,1.0,R,1,0,0,1919,1919-1920,1,2,SEN@QBD,QBD,SEN,2-1
1,1920020045,1,SEN,MTL,1.0,0.0,R,1,0,0,1920,1920-1921,0,1,SEN@MTL,MTL,SEN,1-0
2,1917020025,1,TAN,SEN,3.0,1.0,H,1,0,0,1917,1917-1918,3,1,SEN@TAN,TAN,SEN,3-1
3,1919020003,1,TSP,QBD,7.0,4.0,H,1,0,0,1919,1919-1920,7,4,QBD@TSP,TSP,QBD,7-4
4,1920020025,1,HAM,MTL,6.0,5.0,H,1,0,0,1920,1920-1921,6,5,MTL@HAM,HAM,MTL,6-5


In [2]:
import plotly.graph_objs as go

def plotly_heatmap(res):
    hSet = range(0,max(res.homeScore)+1)
    rSet = range(0,max(res.roadScore)+1)
    z = np.zeros((len(rSet),len(hSet)))
    t = []

    c = res.groupby(['roadScore','homeScore']).gamesPlayed.count()

    for j, r in enumerate(sorted(rSet, reverse = False)):
        for i,h in enumerate(sorted(hSet, reverse = False)):
            try:
                z[j][i] = int(c[j][i])
            except:
                pass
            hov = ("Home: "+str(i)+
                   "<br>Away: "+str(j)+
                   "<br>Count: "+str(int(z[j][i])))
            t.append(hov)

    t = [t[i:i+len(hSet)] for i in range(0, len(t), len(hSet))]


    data = [{
            'z': z,
            'type': 'heatmap',
            'colorscale': [
                [0, 'rgb(255, 255, 255)'],
                [0.0001, 'rgb(230,250,255)'],
                [0.01, 'rgb(150,200,255)'],
                [0.5, 'rgb(150,0,75)'],
                [0.8, 'rgb(120,0,75)'],
                [1., 'rgb(50, 0, 0)']],
            'colorbar': {
                'tick0': 0,
                'tickmode': 'array',
                'tickvals': [0, 500, 1000, 1500, 2000, 2500, 3000, 3500]},
            'hoverinfo':'text',
            'showscale': False,
            'text': t
            },
            go.Histogram(y = res.roadScore,
                         xaxis = 'x2',
                         marker = dict(color = 'rgba(0,0,1,.1)'),
                         hoverinfo = 'text', 
                         text = list(res.groupby('roadScore').gamesPlayed.count())), 
            go.Histogram(x = res.homeScore,
                         yaxis = 'y2',
                         marker = dict(color = 'rgba(0,0,1,.1)'),
                         hoverinfo = 'text', 
                         text = list(res.groupby('homeScore').gamesPlayed.count()))
            ]

    axesColor = 'rgb(200,200,200)'

    layout = go.Layout(
        #title='<b>NHL SCORE DISTRIBUTION</b>',
        titlefont = dict(size = 50, 
                         color = axesColor),
        xaxis = dict(#ticks = list(hSet),
                     domain = [0,.8], 
                     nticks=len(hSet)+1,
                     fixedrange = True,
                     side = 'top',
                     ticklen = 0,
                     tickfont = dict(color = axesColor, size = 15),
                     title='',
                     titlefont=dict(size=18,color=axesColor)),
        yaxis = dict(#ticks= list(rSet),
                     domain = [0.2,1],
                     autorange = 'reversed', 
                     fixedrange = True,
                     nticks=len(rSet)+1,
                     ticklen = 0,
                     tickfont = dict(color = axesColor, size = 15)),
        xaxis2 = dict(zeroline = False,
                      domain = [0.8,1],
                      fixedrange = True,
                      scaleratio = 10,
                      showgrid = False,
                      tickfont = dict(color = axesColor, size = 8),
                      showticklabels=False),
        yaxis2 = dict(zeroline = False,
                      domain = [0,.2],
                      autorange = 'reversed', 
                      fixedrange = True,
                      scaleratio = 10,
                      showgrid = False,
                      tickfont = dict(color = axesColor, size = 8),
                      showticklabels=False),
        annotations = [dict(x=0, y=1.12, xref = 'paper', yref = 'paper',
                            showarrow = False,
                            text = '<b>home<b>', 
                            font = dict(size=25, color = axesColor)),
                       dict(x=-0.09, y=1, xref = 'paper', yref = 'paper',
                            showarrow = False,
                            text = '<b>away<b>',
                            textangle = -90,
                            font = dict(size=25, color = axesColor))],
        showlegend = False,
        hovermode = 'closest',
#       autosize = True,
        height = 550,
        width = 600,
        margin = dict(r=0, b=0),
        )

    fig = go.Figure(data = data, layout = layout)
    
    return fig


In [3]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Event, Input, Output

app = dash.Dash()


app.layout = html.Div([html.H1('NHL Final Scores'),
                  dcc.Graph(id='example-graph',
                            figure = plotly_heatmap(res)
                            )])


In [4]:

if __name__ == '__main__':
    app.run_server(debug=False)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Oct/2018 19:24:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2018 19:24:32] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2018 19:24:32] "GET /_dash-dependencies HTTP/1.1" 200 -
